## 1. Imports


In [ ]:
import os
import sys
import gc
import json
import warnings

import torch
import torch.nn as nn
import torchvision.datasets as datasets
import numpy as np
import matplotlib.pyplot as plt

from tensorboardX import SummaryWriter
from torchvision.transforms import Compose, ToTensor, Resize, Normalize, Lambda
from PIL import PngImagePlugin

from tqdm.auto import tqdm
from IPython.display import clear_output

sys.path.append("..")
from src.enot import SDE, integrate
from src.resnet2 import ResNet_D
from src.cunet import CUNet

from src.tools import set_random_seed, unfreeze, freeze, weights_init_D, fig2tensor
from src.mnistm_utils import MNISTM
from src.plotters import (
    plot_sde_pushed_images,
    plot_sde_pushed_random_class_images,
)
from src.samplers import (
    SubsetGuidedSampler,
    SubsetGuidedDataset,
    get_indicies_subset,
)

LARGE_ENOUGH_NUMBER = 100
PngImagePlugin.MAX_TEXT_CHUNK = LARGE_ENOUGH_NUMBER * (1024**2)

warnings.filterwarnings("ignore")

%matplotlib inline 

In [ ]:
gc.collect()
torch.cuda.empty_cache()

## 2. Config

Dataset choosing in the first rows


In [ ]:
SEED = 0x3060
set_random_seed(SEED)

# dataset choosing
DATASET, DATASET_PATH = "fmnist2mnist", "../datasets/"
# DATASET, DATASET_PATH = "mnist2mnistm", "../datasets/"
# DATASET, DATASET_PATH = "mnist2usps", "../datasets/"
# DATASET, DATASET_PATH = "mnist2kmnist", "../datasets/"

IMG_SIZE = 32
DATASET1_CHANNELS = 1
DATASET2_CHANNELS = 1

# GPU choosing
DEVICE_IDS = [0]
assert torch.cuda.is_available()

CONTINUE = 0

# All hyperparameters below is set to the values used for the experiments, which discribed in the article

# training algorithm settings
BATCH_SIZE = 32
SUBSET_SIZE = 2
NUM_LABELED = 10  # num of labeled target in training set

T_ITERS = 10
MAX_STEPS = 2500 + 1  # 2501 for testing
INTEGRAL_SCALE = 1 / (3 * IMG_SIZE * IMG_SIZE)
EPSILON_SCHEDULER_LAST_ITER = 20000

# optimizer settings
D_LR, T_LR = 1e-4, 1e-4
BETA_D, BETA_T = 0.9, 0.9
T_GRADIENT_MAX_NORM = float(500)
D_GRADIENT_MAX_NORM = float(500)

# SDE network settings
EPSILON = 0  # [0 , 1, 10]
IMAGE_INPUT = True
PREDICT_SHIFT = True
N_STEPS = 5  #
UNET_BASE_FACTOR = 128
TIME_DIM = 128
USE_POSITIONAL_ENCODING = True
ONE_STEP_INIT_ITERS = 0
USE_GRADIENT_CHECKPOINT = False
N_LAST_STEPS_WITHOUT_NOISE = 1

# plot settings
GRAY_PLOTS = True
STEPS_TO_SHOW = 10

# log settings
SMART_INTERVALS = False
INTERVAL_SHRINK_START_TIME = 0.98
TRACK_VAR_INTERVAL = 100
PLOT_INTERVAL = 500
CPKT_INTERVAL = 500

FID_EPOCHS = 1

EXP_NAME = f"ENOT_Class_{DATASET}_{SEED}"
OUTPUT_PATH = f"../saved_models/{EXP_NAME}/"

if not os.path.exists(OUTPUT_PATH):
    os.makedirs(OUTPUT_PATH)

writer = SummaryWriter(f"../logdir/{EXP_NAME}")

In [ ]:
source_subset = torch.tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
new_labels_source = {0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7, 8: 8, 9: 9}
target_subset = torch.tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
new_labels_target = {0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7, 8: 8, 9: 9}

In [ ]:
config = dict(
    SEED=SEED,
    DATASET=DATASET,
    T_ITERS=T_ITERS,
    D_LR=D_LR,
    T_LR=T_LR,
    BATCH_SIZE=BATCH_SIZE,
    SUBSET_SIZE=SUBSET_SIZE,
    UNET_BASE_FACTOR=UNET_BASE_FACTOR,
    N_STEPS=N_STEPS,
    EPSILON=EPSILON,
    USE_POSITIONAL_ENCODING=USE_POSITIONAL_ENCODING,
    TIME_DIM=TIME_DIM,
    INTEGRAL_SCALE=INTEGRAL_SCALE,
    ONE_STEP_INIT_ITERS=ONE_STEP_INIT_ITERS,
    T_GRADIENT_MAX_NORM=T_GRADIENT_MAX_NORM,
    D_GRADIENT_MAX_NORM=D_GRADIENT_MAX_NORM,
    PREDICT_SHIFT=PREDICT_SHIFT,
    SMART_INTERVALS=SMART_INTERVALS,
    INTERVAL_SHRINK_START_TIME=INTERVAL_SHRINK_START_TIME,
    USE_GRADIENT_CHECKPOINT=USE_GRADIENT_CHECKPOINT,
    N_LAST_STEPS_WITHOUT_NOISE=N_LAST_STEPS_WITHOUT_NOISE,
    TRACK_VAR_INTERVAL=TRACK_VAR_INTERVAL,
    EPSILON_SCHEDULER_LAST_ITER=EPSILON_SCHEDULER_LAST_ITER,
    FID_EPOCHS=FID_EPOCHS,
)

with open(os.path.join(OUTPUT_PATH, "config.json"), "w") as json_file:
    json_str = json.dumps(config, indent=4)
    json_file.write(json_str)

log = dict(CONTINUE=CONTINUE)
with open(os.path.join(OUTPUT_PATH, "log.json"), "w") as log_file:
    log_str = json.dumps(log, indent=4)
    log_file.write(log_str)

## 3. Initialize samplers


In [ ]:
source_transform = Compose(
    [
        Resize((IMG_SIZE, IMG_SIZE)),
        ToTensor(),
        Normalize((0.5), (0.5)),
    ]
)
target_transform = source_transform

if DATASET == "mnist2kmnist":
    source = datasets.MNIST
    target = datasets.KMNIST

elif DATASET == "fmnist2mnist":
    source = datasets.FashionMNIST
    target = datasets.MNIST

elif DATASET == "mnist2usps":
    source = datasets.MNIST
    target = datasets.USPS

elif DATASET == "mnist2mnistm":
    DATASET1_CHANNELS = 3
    DATASET2_CHANNELS = 3
    GRAY_PLOTS = False
    source = datasets.MNIST
    target = MNISTM
    source_transform = Compose(
        [
            Resize((IMG_SIZE, IMG_SIZE)),
            ToTensor(),
            Normalize((0.5), (0.5)),
            Lambda(lambda x: -x.repeat(3, 1, 1)),
        ]
    )
    target_transform = Compose(
        [Resize(IMG_SIZE), ToTensor(), Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
    )

In [ ]:
source_train = source(
    root=DATASET_PATH, train=True, download=True, transform=source_transform
)

subset_samples, labels, source_class_indicies = get_indicies_subset(
    source_train,
    new_labels=new_labels_source,
    classes=len(source_subset),
    subset_classes=source_subset,
)
source_train = torch.utils.data.TensorDataset(
    torch.stack(subset_samples), torch.LongTensor(labels)
)


target_train = target(
    root=DATASET_PATH, train=True, download=True, transform=target_transform
)

target_subset_samples, target_labels, target_class_indicies = get_indicies_subset(
    target_train,
    new_labels=new_labels_target,
    classes=len(target_subset),
    subset_classes=target_subset,
)
target_train = torch.utils.data.TensorDataset(
    torch.stack(target_subset_samples), torch.LongTensor(target_labels)
)

train_set = SubsetGuidedDataset(
    source_train,
    target_train,
    num_labeled=NUM_LABELED,
    in_indicies=source_class_indicies,
    out_indicies=target_class_indicies,
)

full_set = SubsetGuidedDataset(
    source_train,
    target_train,
    num_labeled="all",
    in_indicies=source_class_indicies,
    out_indicies=target_class_indicies,
)

In [ ]:
T_XY_sampler = SubsetGuidedSampler(train_set, subsetsize=SUBSET_SIZE)
D_XY_sampler = SubsetGuidedSampler(full_set, subsetsize=1)

In [ ]:
torch.cuda.empty_cache()
gc.collect()
clear_output()

## 4. Training


### Models initialization


In [ ]:
D = ResNet_D(IMG_SIZE, nc=DATASET2_CHANNELS).cuda()
D.apply(weights_init_D)

T = CUNet(
    DATASET1_CHANNELS, DATASET2_CHANNELS, TIME_DIM, base_factor=UNET_BASE_FACTOR
).cuda()

T = SDE(
    shift_model=T,
    epsilon=EPSILON,
    n_steps=N_STEPS,
    time_dim=TIME_DIM,
    n_last_steps_without_noise=N_LAST_STEPS_WITHOUT_NOISE,
    use_positional_encoding=USE_POSITIONAL_ENCODING,
    use_gradient_checkpoint=USE_GRADIENT_CHECKPOINT,
    predict_shift=PREDICT_SHIFT,
    image_input=IMAGE_INPUT,
).cuda()

if len(DEVICE_IDS) > 1 and CONTINUE == 0:
    T = nn.DataParallel(T, device_ids=DEVICE_IDS)
    D = nn.DataParallel(D, device_ids=DEVICE_IDS)

print("T params:", np.sum([np.prod(p.shape) for p in T.parameters()]))
print("D params:", np.sum([np.prod(p.shape) for p in D.parameters()]))

In [ ]:
T_opt = torch.optim.Adam(
    T.parameters(), lr=T_LR, weight_decay=1e-10, betas=(BETA_T, 0.999)
)
D_opt = torch.optim.Adam(
    D.parameters(), lr=D_LR, weight_decay=1e-10, betas=(BETA_D, 0.999)
)
T_scheduler = torch.optim.lr_scheduler.MultiStepLR(
    T_opt, milestones=[15000, 25000, 40000, 55000, 70000], gamma=0.5
)
D_scheduler = torch.optim.lr_scheduler.MultiStepLR(
    D_opt, milestones=[15000, 25000, 40000, 55000, 70000], gamma=0.5
)

if CONTINUE > 0:
    T_opt.load_state_dict(
        torch.load(os.path.join(OUTPUT_PATH, f"T_opt_{SEED}_{CONTINUE}.pt"))
    )
    T_scheduler.load_state_dict(
        torch.load(os.path.join(OUTPUT_PATH, f"T_scheduler_{SEED}_{CONTINUE}.pt"))
    )

    T.load_state_dict(torch.load(os.path.join(OUTPUT_PATH, f"T_{SEED}_{CONTINUE}.pt")))
    D.load_state_dict(torch.load(os.path.join(OUTPUT_PATH, f"D_{SEED}_{CONTINUE}.pt")))

    if len(DEVICE_IDS) > 1:
        T = nn.DataParallel(T, device_ids=DEVICE_IDS)
        D = nn.DataParallel(D, device_ids=DEVICE_IDS)

    D_opt.load_state_dict(
        torch.load(os.path.join(OUTPUT_PATH, f"D_opt_{SEED}_{CONTINUE}.pt"))
    )
    D_scheduler.load_state_dict(
        torch.load(os.path.join(OUTPUT_PATH, f"D_scheduler_{SEED}_{CONTINUE}.pt"))
    )

### Plots Test


In [ ]:
X_fixed, Y_fixed = D_XY_sampler.sample(NUM_LABELED)
X_fixed, Y_fixed = X_fixed.flatten(0, 1), Y_fixed.flatten(0, 1)
print(f"[Debug] {X_fixed.shape=}\n{Y_fixed.shape=}")

In [ ]:
fig, axes = plot_sde_pushed_images(X_fixed, Y_fixed, T, gray=GRAY_PLOTS)
writer.add_image("class images[sde]", fig2tensor(fig))

In [ ]:
fig, axes = plot_sde_pushed_random_class_images(D_XY_sampler, T, gray=GRAY_PLOTS)

### Main training cycle and logging


In [ ]:
gc.collect()
torch.cuda.empty_cache()

In [ ]:
def epsilon_scheduler(step):
    return min(EPSILON, EPSILON * (step / EPSILON_SCHEDULER_LAST_ITER))


progress_bar = tqdm(total=MAX_STEPS, initial=CONTINUE)


for step in range(MAX_STEPS):
    if step < CONTINUE:
        continue

    unfreeze(T)
    freeze(D)

    new_epsilon = epsilon_scheduler(step)
    if len(DEVICE_IDS) > 1:
        T.module.set_epsilon(new_epsilon)
    else:
        T.set_epsilon(new_epsilon)
    # wandb.log({f"Epsilon": new_epsilon}, step=step)

    for t_iter in range(T_ITERS):
        T_opt.zero_grad()

        X0, X1 = T_XY_sampler.sample(BATCH_SIZE)
        # X0.requires_grad_()
        X0 = X0.flatten(0, 1)
        X1 = X1.flatten(0, 1)

        trajectory, times, shifts = T(X0)
        XN = trajectory[:, -1]
        norm = torch.norm(shifts.flatten(start_dim=2), p=2, dim=-1) ** 2
        integral = INTEGRAL_SCALE * integrate(norm, times[0])

        T_loss = (integral + D(X1) - D(XN)).mean()
        writer.add_scalar("T_loss", T_loss.item(), step)
        T_loss.backward()
        T_gradient_norm = torch.nn.utils.clip_grad_norm_(
            T.parameters(), max_norm=T_GRADIENT_MAX_NORM
        )
        T_opt.step()

    # wandb.log({f"T gradient norm": T_gradient_norm.item()}, step=step)
    # wandb.log({f"Mean norm": torch.sqrt(norm).mean().item()}, step=step)
    # wandb.log({f"T_loss": T_loss.item()}, step=step)

    T_scheduler.step()
    del T_loss, X0, X1, XN
    gc.collect()
    torch.cuda.empty_cache()

    freeze(T)
    unfreeze(D)

    D_opt.zero_grad()

    # sample unlabeled Y~Q, X~P
    X0, _ = T_XY_sampler.sample(BATCH_SIZE)
    _, X1 = D_XY_sampler.sample(BATCH_SIZE * SUBSET_SIZE)
    X0 = X0.flatten(0, 1)
    X1 = X1.flatten(0, 1)

    with torch.no_grad():
        trajectory, times, shifts = T(X0)
        XN = trajectory[:, -1]

    norm = torch.norm(shifts.flatten(start_dim=2), p=2, dim=-1) ** 2
    integral = INTEGRAL_SCALE * integrate(norm, times[0])
    D_loss = (-integral - D(X1) + D(XN)).mean()
    writer.add_scalar("D_loss", D_loss.item(), step)
    D_loss.backward()
    D_gradient_norm = torch.nn.utils.clip_grad_norm_(
        D.parameters(), max_norm=D_GRADIENT_MAX_NORM
    )
    D_opt.step()
    D_scheduler.step()

    # wandb.log({f"D gradient norm": D_gradient_norm.item()}, step=step)
    # wandb.log({f"D_loss": D_loss.item()}, step=step)

    # wandb.log({f"integral": integral.mean().item()}, step=step)
    # wandb.log({f"D_X1": D_X1.mean().item()}, step=step)
    # wandb.log({f"D_XN": D_XN.mean().item()}, step=step)
    del D_loss, X0, X1, XN
    gc.collect()
    torch.cuda.empty_cache()

    CONTINUE += 1
    progress_bar.update(1)

    if step % PLOT_INTERVAL == 0:
        clear_output(wait=True)
        progress_bar.close()
        progress_bar = tqdm(total=MAX_STEPS, initial=CONTINUE)
        print("Plotting")

        inference_T = T
        inference_T.eval()
        print("Fixed test Images")
        fig, axes = plot_sde_pushed_images(
            X_fixed, Y_fixed, inference_T, gray=GRAY_PLOTS, plot_trajectory=False
        )
        writer.add_image("Fixed Test Images", fig2tensor(fig), step)
        plt.show(fig)
        plt.close(fig)

        print("Random test Images")
        fig, axes = plot_sde_pushed_random_class_images(
            D_XY_sampler, inference_T, gray=GRAY_PLOTS, plot_trajectory=False
        )
        writer.add_image("Random Test Images", fig2tensor(fig), step)
        plt.show(fig)
        plt.close(fig)

    if step != 0 and step % CPKT_INTERVAL == 0:
        inference_T = T

        inference_T.eval()
        freeze(T)
        if len(DEVICE_IDS) > 1:
            torch.save(
                T.module.state_dict(), os.path.join(OUTPUT_PATH, f"T_{SEED}_{step}.pt")
            )
            torch.save(
                D.module.state_dict(), os.path.join(OUTPUT_PATH, f"D_{SEED}_{step}.pt")
            )
        else:
            torch.save(T.state_dict(), os.path.join(OUTPUT_PATH, f"T_{SEED}_{step}.pt"))
            torch.save(D.state_dict(), os.path.join(OUTPUT_PATH, f"D_{SEED}_{step}.pt"))

        torch.save(
            D_opt.state_dict(), os.path.join(OUTPUT_PATH, f"D_opt_{SEED}_{step}.pt")
        )
        torch.save(
            T_opt.state_dict(), os.path.join(OUTPUT_PATH, f"T_opt_{SEED}_{step}.pt")
        )
        torch.save(
            D_scheduler.state_dict(),
            os.path.join(OUTPUT_PATH, f"D_scheduler_{SEED}_{step}.pt"),
        )
        torch.save(
            T_scheduler.state_dict(),
            os.path.join(OUTPUT_PATH, f"T_scheduler_{SEED}_{step}.pt"),
        )

        log["CONTINUE"] = CONTINUE
        with open(os.path.join(OUTPUT_PATH, "log.json"), "w") as log_file:
            log_str = json.dumps(log, indent=4)
            log_file.write(log_str)
    if step % TRACK_VAR_INTERVAL == 0:
        # after training, using test_transport.ipynb to get fid acc ...
        pass

    gc.collect()
    torch.cuda.empty_cache()

In [ ]:
try:
    writer.close()
    progress_bar.close()
except Exception:
    pass